# Introduction
The following code visualize the [sample data](http://data.apollo.auto/help?name=data_intro_3d&data_key=lidar_obstacle_label&data_type=0&locale=en-us&lang=en) from Apollo.

Note: The visualization image will not show up on the preview of file on Github.

# Requirements

## Python packages
The following packages has to be installed.
- numpy
- numba
- scipy
- pandas
- pyntcloud
- pythreejs

All packages except pyntcloud can be install normally using `pip install`. For pyntcloud, please use

`pip install git+https://github.com/daavoo/pyntcloud`

## Data
The sample point cloud data from Apollo. The extracted folder should be at the same level as this file.

# Extra requirements for this notebook
To actually use this notebook, the following packages are also needed.
- jupyter
- pythreejs

After that, please run the following commands  
`jupyter nbextension install --py --symlink --sys-prefix pythreejs`  
`jupyter nbextension enable --py --sys-prefix pythreejs`  


In [1]:
from dataclasses import dataclass

import numpy as np
import pandas as pd
from pyntcloud import PyntCloud

In [2]:
# See https://stackoverflow.com/a/42366706/10163723
def bounding_box(points, min_x=-np.inf, max_x=np.inf, min_y=-np.inf,
                        max_y=np.inf, min_z=-np.inf, max_z=np.inf):
    """ Compute a bounding_box filter on the given points

    Parameters
    ----------                        
    points: (n,3) array
        The array containing all the points's coordinates. Expected format:
            array([
                [x1,y1,z1],
                ...,
                [xn,yn,zn]])

    min_i, max_i: float
        The bounding box limits for each coordinate. If some limits are missing,
        the default values are -infinite for the min_i and infinite for the max_i.

    Returns
    -------
    bb_filter : boolean array
        The boolean mask indicating wherever a point should be keeped or not.
        The size of the boolean mask will be the same as the number of given points.

    """

    bound_x = np.logical_and(points[:, 0] > min_x, points[:, 0] < max_x)
    bound_y = np.logical_and(points[:, 1] > min_y, points[:, 1] < max_y)
    bound_z = np.logical_and(points[:, 2] > min_z, points[:, 2] < max_z)

    bb_filter = np.logical_and(bound_x, bound_y, bound_z)

    return bb_filter

# Configuration

In [3]:
# Config dtype for numpy.
names = ('x', 'y', 'z', 'i')
offsets = (0, 4, 8, 12)
formats = ('f4', 'f4', 'f4', 'f4')
dt = np.dtype({
    'names': names,
    'offsets': offsets,
    'formats': formats,
}, align=True)

label_names = ('type', 'cx', 'cy', 'cz', 'length', 'width', 'height', 'yaw')
annotation_colors = {
    'vehicle': [255, 0, 0],
    'cyclist': [0, 255, 0],
    'pedestrian': [0, 0, 255],
    'dontCare': [255, 255, 100]
}

# Load point cloud data

In [4]:
# Load point cloud data in all frames.
point_frames = []
for i in range(100):
    frame = pd.DataFrame(np.fromfile('./3d-sample/bin_files/002_%08d.bin' % i, dt))
    
    # Give all points a white color.
    frame['red'] = np.ones(frame.shape[0]) * 255
    frame['green'] = np.ones(frame.shape[0]) * 255
    frame['blue'] = np.ones(frame.shape[0]) * 255
    
    point_frames.append(frame)

## This two code segments below process every frames, so it will take quite some time

In [5]:
@dataclass
class PointCount:
    '''Class for keeping track of point per object.'''
    obj_type: str
    frame: int
    count: int

In [6]:
pedestrian_point_count = [] # type: List[PointCount]

# Load annotation data.
label_frames = []
for i in range(100):
    frame = pd.read_csv('./3d-sample/label_file/002_%08d.bin.txt' % i, sep=' ', header=None, names=label_names)
    
    points = point_frames[i]
    for oidx, obj in frame.iterrows():
        cx = obj['cx'] 
        cy = obj['cy']
        cz = obj['cz']
        width = obj['width']
        length = obj['length']
        height = obj['height']

        min_x = cx - (0.5 * length)
        max_x = cx + (0.5 * length)
        min_y = cy - (0.5 * width)
        max_y = cy + (0.5 * width)
        min_z = cz - (0.5 * height)
        max_z = cz + (0.5 * height)

        bb = bounding_box(points.values, min_x=min_x, max_x=max_x, min_y=min_y,
                            max_y=max_y, min_z=min_z, max_z=max_z)

        #filtered_points = points.loc[bb]
        r, g, b = annotation_colors[obj['type']]
        
        # Color the points within the box with color that is corresponding to the object type.
        points.loc[bb, 'red'], points.loc[bb, 'green'], points.loc[bb, 'blue'] = r, g, b
        
        if obj['type'] == 'pedestrian':
            pc = PointCount('pedestrian', i, len(points.loc[bb, 'red'].index))
            pedestrian_point_count.append(pc)
    
    label_frames.append(frame)

# Statistic Calculation

In [7]:
point_frames[0]

,x,y,z,i,red,green,blue
0,23.570656,9.279344,-2.292419,133.0,255.0,255.0,255.0
1,25.185184,11.014298,-2.352471,144.0,255.0,255.0,255.0
2,16.669695,6.983208,-2.106443,129.0,255.0,255.0,255.0
3,27.448662,13.229650,-2.436704,131.0,255.0,255.0,255.0
4,29.794174,15.765635,-2.539469,162.0,255.0,255.0,255.0
5,17.297304,8.068338,-2.140237,101.0,255.0,255.0,255.0
6,18.156616,9.279870,-2.179916,152.0,255.0,255.0,255.0
7,61.774944,19.087719,-2.283062,255.0,255.0,255.0,255.0
8,59.824184,21.004450,-1.836049,176.0,255.0,255.0,255.0
9,6.424866,1.911165,-0.281108,40.0,255.0,0.0,0.0


Average number of points in a frame is

In [8]:
points_in_frame = np.array([len(f.index) for f in point_frames])
np.average(points_in_frame)

99345.93

In [9]:
pedestrian_point_count

[PointCount(obj_type='pedestrian', frame=0, count=19),
 PointCount(obj_type='pedestrian', frame=0, count=18),
 PointCount(obj_type='pedestrian', frame=0, count=7),
 PointCount(obj_type='pedestrian', frame=0, count=8),
 PointCount(obj_type='pedestrian', frame=0, count=12),
 PointCount(obj_type='pedestrian', frame=1, count=13),
 PointCount(obj_type='pedestrian', frame=1, count=7),
 PointCount(obj_type='pedestrian', frame=1, count=5),
 PointCount(obj_type='pedestrian', frame=2, count=30),
 PointCount(obj_type='pedestrian', frame=2, count=39),
 PointCount(obj_type='pedestrian', frame=2, count=10),
 PointCount(obj_type='pedestrian', frame=4, count=132),
 PointCount(obj_type='pedestrian', frame=4, count=20),
 PointCount(obj_type='pedestrian', frame=5, count=21),
 PointCount(obj_type='pedestrian', frame=5, count=24),
 PointCount(obj_type='pedestrian', frame=6, count=27),
 PointCount(obj_type='pedestrian', frame=7, count=16),
 PointCount(obj_type='pedestrian', frame=7, count=3),
 PointCount(ob

In [10]:
pedestrian_point_counts = np.array([c.count for c in pedestrian_point_count])

In [11]:
len(pedestrian_point_count)

396

Thus, in these 100 frames, there are 396 objects that are labeled as pedestrian.

The average of number of points for each pedestrian is

In [12]:
pedestrian_point_average = np.sum(pedestrian_point_counts) / len(pedestrian_point_count)
pedestrian_point_average

62.89141414141414

In [13]:
print('Min: %.3f' % np.min(pedestrian_point_counts))
print('Max: %.3f' % np.max(pedestrian_point_counts))

Min: 1.000
Max: 959.000


# Displaying the points

- vehicle = red
- cyclist = green
- pedestrian = blue
- dontCare = yellow

In [14]:
points = point_frames[15] # Selecting a frame.

In [15]:
cloud = PyntCloud(points)

Showing the rendered point could. This will not show up in a preview of file in Github website.

In [16]:
cloud.plot(initial_point_size=0.01, backend='pythreejs')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.1779005378484726, 53.93962222337723, 4.133…